# Target Trial Emulation

## Import the necessary libraries

In [2]:
import pandas as pd
import matplotlib as plt

## I. Load the dummy dataset extracted from the package

In [3]:
data_censored = pd.read_csv('../data/data_censored.csv')
data_censored.head()

,id,period,treatment,x1,x2,x3,x4,age,age_s,outcome,censored,eligible
0,1,0,1,1,1.146148,0,0.734203,36,0.083333,0,0,1
1,1,1,1,1,0.002200,0,0.734203,37,0.166667,0,0,0
2,1,2,1,0,-0.481762,0,0.734203,38,0.250000,0,0,0
3,1,3,1,0,0.007872,0,0.734203,39,0.333333,0,0,0
4,1,4,1,1,0.216054,0,0.734203,40,0.416667,0,0,0


In [4]:
data_censored.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725 entries, 0 to 724
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         725 non-null    int64  
 1   period     725 non-null    int64  
 2   treatment  725 non-null    int64  
 3   x1         725 non-null    int64  
 4   x2         725 non-null    float64
 5   x3         725 non-null    int64  
 6   x4         725 non-null    float64
 7   age        725 non-null    int64  
 8   age_s      725 non-null    float64
 9   outcome    725 non-null    int64  
 10  censored   725 non-null    int64  
 11  eligible   725 non-null    int64  
dtypes: float64(3), int64(9)
memory usage: 68.1 KB


In [5]:
data_censored.isna().sum()

id           0
period       0
treatment    0
x1           0
x2           0
x3           0
x4           0
age          0
age_s        0
outcome      0
censored     0
eligible     0
dtype: int64

In [6]:
data_censored.describe()

,id,period,treatment,x1,x2,x3,x4,age,age_s,outcome,censored,eligible
count,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.00000,725.000000
mean,49.278621,7.051034,0.467586,0.405517,-0.173552,0.486897,-0.274722,48.093793,1.091149,0.015172,0.08000,0.234483
std,28.119313,5.802351,0.499293,0.491331,0.997552,0.500173,1.008643,11.834472,0.986206,0.122323,0.27148,0.423968
min,1.000000,0.000000,0.000000,0.000000,-3.284355,0.000000,-3.003087,19.000000,-1.333333,0.000000,0.00000,0.000000
25%,23.000000,2.000000,0.000000,0.000000,-0.809344,0.000000,-0.861899,40.000000,0.416667,0.000000,0.00000,0.000000
50%,50.000000,6.000000,0.000000,0.000000,-0.163060,0.000000,-0.316594,49.000000,1.166667,0.000000,0.00000,0.000000
75%,73.000000,12.000000,1.000000,1.000000,0.494103,1.000000,0.299510,56.000000,1.750000,0.000000,0.00000,0.000000
max,99.000000,19.000000,1.000000,1.000000,3.907648,1.000000,2.048087,78.000000,3.583333,1.000000,1.00000,1.000000
